In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import nltk
import emoji

from googletrans import Translator
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from wordcloud import WordCloud
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from transformers import pipeline, AutoTokenizer
from collections import Counter


%matplotlib inline
sns.set_style("whitegrid")

#set warning 
import warnings
warnings.filterwarnings('ignore')


pd.pandas.set_option('display.max_columns', None)

In [22]:
filename = "BYD-M6.csv"
df = pd.read_csv(filename, encoding = 'latin-1')
df

,conversation_id_str,created_at,favorite_count,full_text,id_str,image_url,in_reply_to_screen_name,lang,location,quote_count,reply_count,retweet_count,tweet_url,user_id_str,username
0,1956653409953886320,Sat Aug 16 09:45:00 +0000 2025,0,Sepanjang 2025 20.795 unit mobil CBU dari BYD ...,1956653409953886320,NaN,NaN,in,NaN,0,0,0,https://x.com/undefined/status/195665340995388...,350668803,NaN
1,1956214303339487523,Fri Aug 15 11:17:06 +0000 2025,0,@tanyarlfes Motor impian Honda super cub C100 ...,1956314202072650209,NaN,tanyarlfes,in,NaN,0,0,0,https://x.com/undefined/status/195631420207265...,1193997693904809984,NaN
2,1956020121518137693,Thu Aug 14 15:48:32 +0000 2025,0,tiap pagi n sore lagi melintasi jalanan sudirm...,1956020121518137693,NaN,NaN,in,NaN,0,0,0,https://x.com/undefined/status/195602012151813...,568999509,NaN
3,1952613025418252500,Tue Aug 05 06:09:57 +0000 2025,0,Di tengah mahalnya harga mobil baru dan melonj...,1952613025418252500,https://pbs.twimg.com/media/GxkTGvTbQAER0u8.jpg,NaN,in,NaN,0,0,0,https://x.com/undefined/status/195261302541825...,1912027937676853249,NaN
4,1951489959233573009,Sat Aug 02 03:47:17 +0000 2025,0,BYD M6: Kombinasi Desain Elegan dan Kekuatan M...,1951489959233573009,NaN,NaN,in,NaN,0,0,0,https://x.com/undefined/status/195148995923357...,227969034,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,25885078568,Wed Sep 29 13:59:53 +0000 2010,0,BYD Automobile Cina Pasarkan Mobil Keluarga M6...,25885078568,NaN,NaN,in,NaN,0,0,0,https://x.com/undefined/status/25885078568,149796787,NaN
340,25885078530,Wed Sep 29 13:59:53 +0000 2010,0,BYD Automobile Cina Pasarkan Mobil Keluarga M6...,25885078530,NaN,NaN,in,NaN,0,0,0,https://x.com/undefined/status/25885078530,170540140,NaN
341,25885078003,Wed Sep 29 13:59:52 +0000 2010,0,BYD Automobile Cina Pasarkan Mobil Keluarga M6...,25885078003,NaN,NaN,in,NaN,0,0,1,https://x.com/undefined/status/25885078003,18129942,NaN
342,25885077287,Wed Sep 29 13:59:52 +0000 2010,0,BYD Automobile Cina Pasarkan Mobil Keluarga M6...,25885077287,NaN,NaN,in,NaN,0,0,0,https://x.com/undefined/status/25885077287,20963426,NaN


# DELETE COLOMN

In [23]:
df = df.drop(columns=[
    'conversation_id_str',
    'created_at',
    'favorite_count',
    'id_str',
    'image_url',
    'in_reply_to_screen_name',
    'lang',
    'location',
    'quote_count',
    'reply_count',
    'retweet_count',
    'tweet_url',
    'user_id_str',
    'username'
])

df.rename(columns={"full_text" : "Comment"}, inplace=True)

df


,Comment
0,Sepanjang 2025 20.795 unit mobil CBU dari BYD ...
1,@tanyarlfes Motor impian Honda super cub C100 ...
2,tiap pagi n sore lagi melintasi jalanan sudirm...
3,Di tengah mahalnya harga mobil baru dan melonj...
4,BYD M6: Kombinasi Desain Elegan dan Kekuatan M...
...,...
339,BYD Automobile Cina Pasarkan Mobil Keluarga M6...
340,BYD Automobile Cina Pasarkan Mobil Keluarga M6...
341,BYD Automobile Cina Pasarkan Mobil Keluarga M6...
342,BYD Automobile Cina Pasarkan Mobil Keluarga M6...


# STOPWORD, STEMMING, AND REMOVING EMOJIS

In [24]:
# Indonesian stopwords
indonesian_stopwords = set(nltk.corpus.stopwords.words('indonesian'))

# You can add more custom stopwords if needed (bisa ditambah)
custom_stopwords = set(['di', 'yang', 'dan', 'ini', 'dengan', 'dari', 'untuk', 
 'pada', 'jadi', 'bisa', 'ada', 'buat', 'lebih', 'sama', 'juga', 'lagi', 'ya']
)
all_stopwords = indonesian_stopwords.union(custom_stopwords)


# Initialize the Indonesian stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Stemming
prefixes = ("me", "mem", "men", "meng", "meny",
             "ber", "ter", "se", "ke", "pe", "di")

# Function to remove URLs
def remove_urls(text):
    return re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)


# Function to remove emojis
def remove_emojis(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)


# Function to remove non-alphabetic characters, URLs, and emojis, tokenize, remove stopwords, and stem (kata dibawah 3 diatas 20)
def preprocess_text(text):
    text = str(text)
    text = remove_urls(text)  # Remove URLs
    text = remove_emojis(text)  # Remove emojis
    text = re.sub(r'[^A-Za-z\s]', '', text)  # Remove non-alphabetic characters
    tokens = word_tokenize(text)  # Tokenize the text
    filtered_tokens = [stemmer.stem(token) for token in tokens if 3 <= len(token) <= 20 and token.lower() not in all_stopwords]  # Remove stopwords and apply stemming
    return ' '.join(filtered_tokens)


# Apply preprocessing to the text column
df['Cleaned_Comment'] = df['Comment'].apply(preprocess_text)

# Display the first few rows of the dataframe with cleaned text
df_display = df[['Comment', 'Cleaned_Comment']]
df_display.head()

KeyboardInterrupt: 

# TOKENIZATION

In [ ]:
# Model IndoBERT Sentiment
model_name = 'mdhugol/indonesia-bert-sentiment-classification'

# Gunakan PyTorch backend
sentiment_pipeline = pipeline('sentiment-analysis', model=model_name, tokenizer=model_name, framework='pt')
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Truncate teks (maks 512 token)
def truncate_text(text, max_length=512):
    inputs = tokenizer(text, max_length=max_length, truncation=True, return_tensors='pt')
    return tokenizer.decode(inputs['input_ids'][0], skip_special_tokens=True)

# Preprocessing dan analisis sentimen
df['Cleaned_Comment'] = df['Comment'].apply(preprocess_text)
df['Sentiment'] = df['Cleaned_Comment'].apply(lambda x: sentiment_pipeline(truncate_text(x))[0]['label'])

# Tampilkan hasil
df_display = df[['Comment', 'Cleaned_Comment', 'Sentiment']]
df_display.head()


ValueError: Could not load model mdhugol/indonesia-bert-sentiment-classification with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForSequenceClassification'>, <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'>). See the original errors:

while loading with AutoModelForSequenceClassification, an error is thrown:
Traceback (most recent call last):
  File "c:\Users\edlyn\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\pipelines\base.py", line 293, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
  File "c:\Users\edlyn\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\auto\auto_factory.py", line 604, in from_pretrained
    return model_class.from_pretrained(
  File "c:\Users\edlyn\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\modeling_utils.py", line 277, in _wrapper
    return func(*args, **kwargs)
  File "

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

# Plot the distribution of sentiment labels
plt.figure(figsize=(10, 6))
sentiment_counts = df['Sentiment'].value_counts()
sns.barplot(x=sentiment_counts.index, y=sentiment_counts.values, palette='viridis')


plt.title('Distribution of Sentiment Labels')
plt.xlabel('Sentiment')
plt.ylabel('Number of Reviews')
plt.xticks(rotation=0)
plt.show()